# Classifier Tests with extracted data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import extractor as ext

In [3]:
fbase = pd.read_csv('./1_filtered.csv', header=None)
fbase.head()

,0,1,2,3
0,8.456798,-4.466378,7.215951,0.019221
1,8.454815,-4.464132,7.212556,0.019295
2,8.453542,-4.461002,7.207107,0.019680
3,8.449433,-4.457170,7.201194,0.020556
4,8.440568,-4.453560,7.195996,0.020409


In [4]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
len(cycles)

4

In [5]:
# First Sensor, First cycle
cycles[0][0].grip.head()

108000    0.037666
108001    0.012989
108002   -0.025789
108003   -0.062451
108004   -0.078099
Name: 0, dtype: float64

In [6]:
# First Sensor, First cycle
cycles[0][0].flexion.head()

48000   -0.076478
48001   -0.076115
48002   -0.048953
48003   -0.003499
48004    0.039958
Name: 0, dtype: float64

# Generate Dataset based on feature extractors

1. SampEn - Sample Entropy
2. RMS - Root Mean Square
3. WL - Waveform Length
4. WAMP - Willison Amplitude
5. ApEn - Approximate entropy
6. MAV - Mean Absolute Value

The data will be offsetted with 100ms.
Each cycle will be interpreted as a new data acquisition
3 Datasets, one with 299ms, other with 500ms, other with 1 second.

Dataset 299ms:
| SENSOR_0_RMS   |  SENSOR_1_RMS | SENSOR_2_RMS | SENSOR_3_RMS| SENSOR_0_SAMPEN   |  SENSOR_1_SAMPEN | SENSOR_2_SAMPEN | SENSOR_3_SAMPEN |...| CLASS | 
| --- | ---| ---| ---| ---| ---| ---| ---| ---| ---|
| 12 | 13 | 45 | 4 | 5 | 23 | 123 | 123|...|GRIP|


TODO: Do we use the onset algorithm?

IDEIA:

Usar dois dados rápidos para análise rápida e depois no meio do movimento da mão verificar a análise com mais tempo e corrigir o movimento

In [7]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
np.array(cycles).shape

(4, 5)

In [8]:
import feature_extractors as fe

In [42]:
def populatedict(data, name, dic:dict):
    data = np.array(data)
    if not dic.get(name+"_RMS"): dic[name+"_RMS"] = []
    if not dic.get(name+"_WAVELEN"): dic[name+"_WAVELEN"] = []
    if not dic.get(name+"_WAMP"): dic[name+"_WAMP"] = []
    if not dic.get(name+"_APPEN"): dic[name+"_APPEN"] = []
    if not dic.get(name+"_SAMPEN"): dic[name+"_SAMPEN"] = []
    if not dic.get(name+"_MAV"): dic[name+"_MAV"] = []

    dic[name+"_RMS"].append(fe.rms(data))
    dic[name+"_WAVELEN"].append(fe.waveformlen(data))
    dic[name+"_WAMP"].append(fe.wamp(data))
    dic[name+"_APPEN"].append(fe.app_entropy(data))
    dic[name+"_SAMPEN"].append(fe.sampen(data))
    dic[name+"_MAV"].append(fe.mav(data))

In [43]:
newds = {}
newds_offsetted = {}
newds["class"] = []
newds_offsetted["class"] = []

for j in range(4):
    for i in range(5):
        lower_limit = int(extractor.to_ms(100))
        upper_limit = int(extractor.to_ms(299)) # TODO: Create two databases, one with the offset and the other without

        populatedict(cycles[j][i].rest[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].rest[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].extension[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].extension[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].flexion[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].flexion[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].ulnar_deviation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].ulnar_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].radial_deviation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].radial_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].grip[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].grip[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].finger_abduction[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].finger_abduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].finger_adduction[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].finger_adduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].supination[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].supination[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

        populatedict(cycles[j][i].pronation[:upper_limit], "SENSOR" + str(j), newds)
        populatedict(cycles[j][i].pronation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted)

for i in range(5):
    newds["class"].append("rest")
    newds_offsetted["class"].append("rest")
    
for i in range(5):
    newds["class"].append("extension")
    newds_offsetted["class"].append("extension")

for i in range(5):
    newds["class"].append("flexion")
    newds_offsetted["class"].append("flexion")

for i in range(5):
    newds["class"].append("ulnar_deviation")
    newds_offsetted["class"].append("ulnar_deviation")

for i in range(5):
    newds["class"].append("radial_deviation")
    newds_offsetted["class"].append("radial_deviation")

for i in range(5):
    newds["class"].append("grip")
    newds_offsetted["class"].append("grip")

for i in range(5):
    newds["class"].append("finger_abduction")
    newds_offsetted["class"].append("finger_abduction")

for i in range(5):
    newds["class"].append("finger_adduction")
    newds_offsetted["class"].append("finger_adduction")

for i in range(5):
    newds["class"].append("supination")
    newds_offsetted["class"].append("supination")

for i in range(5):
    newds["class"].append("pronation")
    newds_offsetted["class"].append("pronation")

print(len(newds["class"]))
print(len(newds["SENSOR0_MAV"]))

10
10


In [44]:
pd.DataFrame(newds).head(10)

,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.031721,5.290727,183,0.687755,0.673635,0.022882,0.032332,6.344441,239,...,33,0.827534,0.790555,0.008300,0.006577,1.331569,14,0.692217,0.694896,0.004631
1,rest,0.070548,11.157630,285,0.580385,0.467871,0.044598,0.087475,13.830049,287,...,138,0.507690,0.379388,0.021779,0.005762,1.052785,3,0.665619,0.592555,0.004100
2,rest,0.027661,5.202771,175,0.796447,0.824434,0.021468,0.030158,5.347072,186,...,22,0.874635,0.954179,0.007136,0.004524,0.902125,0,0.653822,0.693147,0.003245
3,rest,0.149778,27.378830,495,0.736398,0.684526,0.102145,0.164398,32.458609,495,...,349,0.798038,0.637110,0.051059,0.011593,2.186982,43,0.692162,0.664341,0.008453
4,rest,0.028181,5.235600,202,0.758380,0.782492,0.021767,0.029030,6.232415,252,...,35,0.911575,0.969754,0.008016,0.005456,1.148297,2,0.739630,0.772824,0.004146
5,extension,0.169467,32.219861,513,0.765109,0.757301,0.127672,0.202065,44.123633,519,...,400,0.732960,0.689266,0.067502,0.013473,2.498572,56,0.684669,0.678868,0.010139
6,extension,0.030378,5.677460,231,0.788445,0.807206,0.024035,0.029124,6.050261,237,...,50,0.951872,0.980597,0.008824,0.005920,1.143573,10,0.610414,0.617949,0.003884
7,extension,0.144170,23.205370,406,0.581151,0.443941,0.093274,0.127309,25.785347,438,...,263,0.608826,0.468838,0.041116,0.010748,1.820793,29,0.628951,0.573245,0.007446
8,extension,0.028181,5.235600,202,0.758380,0.782492,0.021767,0.038317,6.934336,246,...,23,0.892844,0.872415,0.009249,0.005690,1.148667,2,0.694973,0.723309,0.004217
9,extension,0.169467,32.219861,513,0.765109,0.757301,0.127672,0.255301,63.773884,545,...,481,0.861830,0.903799,0.076756,0.019494,3.839257,133,0.717062,0.742920,0.015384


In [45]:
pd.DataFrame(newds_offsetted).head(10)


,class,SENSOR0_RMS,SENSOR0_WAVELEN,SENSOR0_WAMP,SENSOR0_APPEN,SENSOR0_SAMPEN,SENSOR0_MAV,SENSOR1_RMS,SENSOR1_WAVELEN,SENSOR1_WAMP,...,SENSOR2_WAMP,SENSOR2_APPEN,SENSOR2_SAMPEN,SENSOR2_MAV,SENSOR3_RMS,SENSOR3_WAVELEN,SENSOR3_WAMP,SENSOR3_APPEN,SENSOR3_SAMPEN,SENSOR3_MAV
0,rest,0.025778,4.690946,152,0.761876,0.765203,0.018910,0.028474,5.824146,213,...,24,0.817085,0.769114,0.007545,0.006339,1.351022,6,0.750815,0.806641,0.004854
1,rest,0.114155,19.795009,404,0.722534,0.461799,0.082869,0.132646,25.435204,403,...,270,0.654284,0.347822,0.040934,0.008554,1.499724,15,0.653104,0.511161,0.006082
2,rest,0.027076,5.018998,176,0.783307,0.828326,0.021332,0.026556,4.955964,172,...,17,0.887917,0.897942,0.007100,0.004261,0.819225,0,0.662672,0.655396,0.002874
3,rest,0.187945,36.668787,535,0.794588,0.804381,0.140118,0.185544,44.149425,538,...,443,0.774506,0.790013,0.075245,0.016339,3.208140,91,0.683923,0.708447,0.012338
4,rest,0.026002,4.902221,184,0.784960,0.822984,0.019759,0.027063,5.969223,247,...,35,0.946782,1.023901,0.007522,0.005719,1.239497,5,0.706105,0.735733,0.004265
5,extension,0.227313,46.181424,537,0.822075,0.872817,0.179824,0.234101,58.053645,539,...,480,0.824160,0.837017,0.086540,0.017980,3.515055,116,0.699177,0.716088,0.014377
6,extension,0.027373,5.193144,213,0.822551,0.829787,0.021753,0.028234,5.880219,234,...,44,1.033309,1.090219,0.008779,0.006723,1.287608,15,0.599604,0.558602,0.004497
7,extension,0.225400,45.008615,528,0.744666,0.665789,0.165790,0.208578,46.793188,532,...,407,0.723768,0.646587,0.073664,0.016956,2.951897,84,0.679359,0.636540,0.012414
8,extension,0.026002,4.902221,184,0.784960,0.822984,0.019759,0.037966,6.860792,233,...,32,0.853236,0.814000,0.009403,0.005639,1.152742,1,0.691431,0.724308,0.004262
9,extension,0.227313,46.181424,537,0.822075,0.872817,0.179824,0.285707,77.401970,563,...,509,0.851822,0.947739,0.091198,0.022720,4.927402,199,0.757192,0.809399,0.018717


In [ ]:
import numpy as np

import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error